In [33]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [34]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."


import tensorboard
tensorboard.__version__

TensorFlow version:  2.16.1


'2.16.2'

In [35]:
import pandas as pd
import ast

data_path = '../paired_data.csv'

data = pd.read_csv(data_path)
data["overview_embedding"] = data["overview_embedding"].apply(ast.literal_eval)

In [36]:
from sentence_transformers import SentenceTransformer
transformer = SentenceTransformer('all-MiniLM-L6-v2')

/home/matt/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [37]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from joblib import dump

# Convert embedding column from list to numpy array
data["overview_embedding"] = data["overview_embedding"].apply(np.array)

# Split the data into features (X) and target (y)
X = np.vstack(data["overview_embedding"].values)
y = data['average_rating']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

# Standardize features
x_scaler = StandardScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled = x_scaler.transform(X_test)

# Define the neural network architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_fn = tf.keras.losses.MeanSquaredError(name="mean_squared_error")
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Checkpoint callback
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Train the model
model.fit(X_train_scaled, y_train, epochs=7, validation_split=0.2, callbacks=[checkpoint_callback, tensorboard_callback])

# Evaluate the model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

# Save the model
dump(x_scaler, 'scaler.joblib')
model.save('movie_rating_prediction_model.keras')

Epoch 1/7


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 384), dtype=float32). Expected shape (None, 35091, 384), but input has incompatible shape (None, 384)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 384), dtype=float32)
  • training=True
  • mask=None

In [ ]:
from sklearn.preprocessing import StandardScaler
from joblib import load
import numpy as np

overview: str = input("Give your movie overview: ")
print(overview)

# Now you can use the loaded scaler to transform new data
scaled_embed = x_scaler.transform(np.array(transformer.encode(overview)).reshape(1, -1))

predicted_rating = model.predict(scaled_embed)

print(f"prediction: {predicted_rating}")

In a desolate town stands a decrepit mansion, its once majestic halls now cloaked in decay and silence. Ignored by locals, it holds secrets that refuse to stay buried. Determined to unravel its mysteries, a group of reckless teenagers led by Sarah, an adventurous soul with a penchant for danger, ventures inside one moonlit night.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
prediction: [[3.080523]]
